# Notebook 74: 行列で並列化 ― ベクトル版逆伝播

## Matrix Backpropagation: Vectorized Implementation

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第5章として、スカラー版の逆伝播を **行列演算** に拡張し、バッチ処理を可能にします。

### 学習目標

1. **行列微分** の基礎（ヤコビアン）を理解する
2. **バッチ処理** における勾配計算を実装する
3. **Linear層** と **活性化層** をクラスとして設計する
4. **多層パーセプトロン（MLP）** を NumPy のみで構築する

### 前提知識

- Notebook 70-73 の内容
- 行列演算（行列積、転置）の基礎

---

## 目次

1. [スカラーから行列へ：次元の整理](#1-スカラーから行列へ次元の整理)
2. [行列微分の基礎](#2-行列微分の基礎)
3. [Linear層の実装](#3-linear層の実装)
4. [活性化層の実装](#4-活性化層の実装)
5. [損失関数の実装](#5-損失関数の実装)
6. [多層パーセプトロンの構築](#6-多層パーセプトロンの構築)
7. [勾配チェック](#7-勾配チェック)
8. [演習問題](#8-演習問題)
9. [まとめと次のステップ](#9-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. スカラーから行列へ：次元の整理

### 1.1 バッチ処理の必要性

実際のニューラルネットワークでは、複数のサンプルを **同時に** 処理します（バッチ処理）。

| スカラー版 | 行列版 |
|-----------|--------|
| 1サンプルずつ処理 | N サンプル同時処理 |
| for ループが必要 | 行列演算で一括 |
| 遅い | 高速（SIMD/GPU活用） |

### 1.2 次元の規約

```
入力 X:  (N, D_in)   - N: バッチサイズ, D_in: 入力次元
重み W:  (D_in, D_out) - D_out: 出力次元
バイアス b: (D_out,)   - ブロードキャスト
出力 Y:  (N, D_out)
```

In [ ]:
# 次元の確認
N = 4       # バッチサイズ
D_in = 3    # 入力次元
D_out = 2   # 出力次元

# ダミーデータ
X = np.random.randn(N, D_in)
W = np.random.randn(D_in, D_out)
b = np.random.randn(D_out)

# 線形変換: Y = XW + b
Y = X @ W + b  # @ は行列積

print("【次元の確認】")
print(f"X (入力):    {X.shape} = (バッチサイズ, 入力次元)")
print(f"W (重み):    {W.shape} = (入力次元, 出力次元)")
print(f"b (バイアス): {b.shape} = (出力次元,)")
print(f"Y (出力):    {Y.shape} = (バッチサイズ, 出力次元)")

print(f"\n【行列積の計算】")
print(f"X @ W: ({N}, {D_in}) @ ({D_in}, {D_out}) → ({N}, {D_out})")

---

## 2. 行列微分の基礎

### 2.1 スカラー関数の行列微分

損失 $L$ はスカラー値です。$L$ を行列 $W$ で微分すると、$W$ と同じ形状の行列が得られます：

$$
\frac{\partial L}{\partial W} \in \mathbb{R}^{D_{in} \times D_{out}}
$$

### 2.2 線形層の勾配導出

線形変換 $Y = XW + b$ の勾配を考えます。

上流から $\frac{\partial L}{\partial Y}$ が来たとき：

$$
\frac{\partial L}{\partial W} = X^T \frac{\partial L}{\partial Y}
$$

$$
\frac{\partial L}{\partial b} = \sum_{i=1}^{N} \frac{\partial L}{\partial Y_i} = \mathbf{1}^T \frac{\partial L}{\partial Y}
$$

$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^T
$$

In [ ]:
# 行列微分の導出を確認
print("【線形層 Y = XW + b の勾配】\n")

print("上流からの勾配: dL/dY の形状 = (N, D_out)")
print("")
print("1. dL/dW = X^T @ dL/dY")
print(f"   ({D_in}, {N}) @ ({N}, {D_out}) → ({D_in}, {D_out})")
print("")
print("2. dL/db = sum(dL/dY, axis=0)")
print(f"   ({N}, {D_out}) → ({D_out},)")
print("")
print("3. dL/dX = dL/dY @ W^T")
print(f"   ({N}, {D_out}) @ ({D_out}, {D_in}) → ({N}, {D_in})")

---

## 3. Linear層の実装

### 3.1 Layer基底クラス

In [ ]:
class Layer(ABC):
    """
    ニューラルネットワーク層の基底クラス
    """
    
    def __init__(self):
        self.params = {}    # 学習パラメータ
        self.grads = {}     # パラメータの勾配
        self.cache = {}     # 逆伝播用のキャッシュ
    
    @abstractmethod
    def forward(self, x):
        """順伝播"""
        pass
    
    @abstractmethod
    def backward(self, dout):
        """逆伝播: 上流の勾配 dout を受け取り、下流への勾配を返す"""
        pass


print("Layer基底クラスを定義しました")

### 3.2 Linear層（全結合層）

In [ ]:
class Linear(Layer):
    """
    全結合層（線形変換）: Y = XW + b
    
    Args:
        in_features: 入力次元
        out_features: 出力次元
    """
    
    def __init__(self, in_features, out_features):
        super().__init__()
        
        # Xavier初期化
        scale = np.sqrt(2.0 / (in_features + out_features))
        self.params['W'] = np.random.randn(in_features, out_features) * scale
        self.params['b'] = np.zeros(out_features)
        
        self.grads['W'] = None
        self.grads['b'] = None
    
    def forward(self, x):
        """
        順伝播: Y = XW + b
        
        Args:
            x: 入力 (N, in_features)
        Returns:
            出力 (N, out_features)
        """
        self.cache['x'] = x  # 逆伝播で使う
        return x @ self.params['W'] + self.params['b']
    
    def backward(self, dout):
        """
        逆伝播
        
        Args:
            dout: 上流からの勾配 (N, out_features)
        Returns:
            下流への勾配 (N, in_features)
        """
        x = self.cache['x']
        
        # 勾配の計算
        self.grads['W'] = x.T @ dout          # (in, N) @ (N, out) = (in, out)
        self.grads['b'] = np.sum(dout, axis=0) # (out,)
        
        # 下流への勾配
        dx = dout @ self.params['W'].T         # (N, out) @ (out, in) = (N, in)
        
        return dx


# テスト
linear = Linear(3, 2)
x = np.random.randn(4, 3)  # バッチサイズ4, 入力次元3

# 順伝播
y = linear.forward(x)
print(f"【Linear層テスト】")
print(f"入力 x: {x.shape}")
print(f"出力 y: {y.shape}")

# 逆伝播
dout = np.ones_like(y)  # 上流からの勾配（仮）
dx = linear.backward(dout)
print(f"\n逆伝播:")
print(f"dL/dW: {linear.grads['W'].shape}")
print(f"dL/db: {linear.grads['b'].shape}")
print(f"dL/dx: {dx.shape}")

---

## 4. 活性化層の実装

### 4.1 Sigmoid層

In [ ]:
class Sigmoid(Layer):
    """
    シグモイド活性化関数: y = 1 / (1 + exp(-x))
    
    逆伝播: dx = dout * y * (1 - y)
    """
    
    def forward(self, x):
        # 数値安定性のための実装
        y = np.where(
            x >= 0,
            1 / (1 + np.exp(-x)),
            np.exp(x) / (1 + np.exp(x))
        )
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * y * (1 - y)


# テスト
sigmoid = Sigmoid()
x = np.array([[-1, 0, 1], [2, -2, 0]])
y = sigmoid.forward(x)
print(f"Sigmoid forward: x.shape={x.shape}, y.shape={y.shape}")
print(f"y = \n{y}")

### 4.2 ReLU層

In [ ]:
class ReLU(Layer):
    """
    ReLU活性化関数: y = max(0, x)
    
    逆伝播: dx = dout * (x > 0)
    """
    
    def forward(self, x):
        self.cache['mask'] = (x > 0).astype(float)
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * self.cache['mask']


# テスト
relu = ReLU()
x = np.array([[-1, 0, 1], [2, -2, 0.5]])
y = relu.forward(x)
print(f"ReLU forward:")
print(f"x = \n{x}")
print(f"y = \n{y}")

### 4.3 Tanh層

In [ ]:
class Tanh(Layer):
    """
    Tanh活性化関数: y = tanh(x)
    
    逆伝播: dx = dout * (1 - y²)
    """
    
    def forward(self, x):
        y = np.tanh(x)
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * (1 - y ** 2)


print("活性化層を定義しました")

---

## 5. 損失関数の実装

### 5.1 MSE損失

In [ ]:
class MSELoss:
    """
    平均二乗誤差損失: L = (1/N) * Σ(y - t)²
    
    逆伝播: dL/dy = (2/N) * (y - t)
    """
    
    def __init__(self):
        self.cache = {}
    
    def forward(self, y, t):
        """
        Args:
            y: 予測値 (N, D)
            t: 正解値 (N, D)
        Returns:
            損失（スカラー）
        """
        self.cache['y'] = y
        self.cache['t'] = t
        self.cache['N'] = y.shape[0]
        
        loss = np.mean((y - t) ** 2)
        return loss
    
    def backward(self):
        """
        Returns:
            dL/dy (N, D)
        """
        y = self.cache['y']
        t = self.cache['t']
        N = self.cache['N']
        
        return (2 / N) * (y - t)


# テスト
mse = MSELoss()
y = np.array([[0.5, 0.8], [0.3, 0.9]])
t = np.array([[1.0, 1.0], [0.0, 1.0]])

loss = mse.forward(y, t)
grad = mse.backward()
print(f"MSE Loss: {loss:.6f}")
print(f"勾配 dL/dy:\n{grad}")

### 5.2 CrossEntropy損失（Softmax + CrossEntropy）

In [ ]:
class SoftmaxCrossEntropy:
    """
    Softmax + CrossEntropy 損失（分類問題用）
    
    Softmax: p_i = exp(x_i) / Σexp(x_j)
    CrossEntropy: L = -Σ t_i * log(p_i)
    
    逆伝播: dL/dx = p - t（シンプルな形！）
    """
    
    def __init__(self):
        self.cache = {}
    
    def _softmax(self, x):
        """数値安定性を考慮したSoftmax"""
        x_shifted = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x_shifted)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, x, t):
        """
        Args:
            x: ロジット (N, C) - Softmax前の値
            t: 正解ラベル (N, C) - one-hot または (N,) - クラスインデックス
        Returns:
            損失（スカラー）
        """
        N = x.shape[0]
        
        # one-hot変換（必要な場合）
        if t.ndim == 1:
            t_onehot = np.zeros_like(x)
            t_onehot[np.arange(N), t] = 1
            t = t_onehot
        
        p = self._softmax(x)
        self.cache['p'] = p
        self.cache['t'] = t
        self.cache['N'] = N
        
        # クロスエントロピー: -Σ t * log(p)
        loss = -np.sum(t * np.log(p + 1e-10)) / N
        return loss
    
    def backward(self):
        p = self.cache['p']
        t = self.cache['t']
        N = self.cache['N']
        
        # dL/dx = (p - t) / N
        return (p - t) / N


# テスト
sce = SoftmaxCrossEntropy()
x = np.array([[2.0, 1.0, 0.1], [0.5, 2.5, 0.3]])
t = np.array([0, 1])  # クラスインデックス

loss = sce.forward(x, t)
grad = sce.backward()
print(f"CrossEntropy Loss: {loss:.6f}")
print(f"Softmax出力:\n{sce.cache['p']}")
print(f"勾配 dL/dx:\n{grad}")

---

## 6. 多層パーセプトロンの構築

### 6.1 MLPクラス

In [ ]:
class MLP:
    """
    多層パーセプトロン（Multi-Layer Perceptron）
    
    NumPyのみで構築した、順伝播・逆伝播が可能なニューラルネットワーク
    """
    
    def __init__(self, layer_dims, activation='relu'):
        """
        Args:
            layer_dims: 各層の次元リスト [入力, 隠れ1, 隠れ2, ..., 出力]
            activation: 活性化関数 ('relu', 'sigmoid', 'tanh')
        """
        self.layers = []
        
        # 活性化関数の選択
        activation_class = {
            'relu': ReLU,
            'sigmoid': Sigmoid,
            'tanh': Tanh
        }[activation]
        
        # 層の構築
        for i in range(len(layer_dims) - 1):
            # 線形層
            self.layers.append(Linear(layer_dims[i], layer_dims[i+1]))
            
            # 最後の層以外は活性化関数を追加
            if i < len(layer_dims) - 2:
                self.layers.append(activation_class())
    
    def forward(self, x):
        """順伝播"""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, dout):
        """逆伝播"""
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout
    
    def get_params(self):
        """全パラメータを取得"""
        params = []
        for layer in self.layers:
            if hasattr(layer, 'params'):
                for name, param in layer.params.items():
                    params.append((layer, name, param))
        return params
    
    def get_grads(self):
        """全勾配を取得"""
        grads = []
        for layer in self.layers:
            if hasattr(layer, 'grads'):
                for name, grad in layer.grads.items():
                    grads.append((layer, name, grad))
        return grads
    
    def zero_grad(self):
        """勾配をゼロにリセット"""
        for layer in self.layers:
            if hasattr(layer, 'grads'):
                for name in layer.grads:
                    layer.grads[name] = None


# テスト
mlp = MLP([3, 4, 2], activation='relu')
print(f"【MLP構造】")
for i, layer in enumerate(mlp.layers):
    print(f"  Layer {i}: {layer.__class__.__name__}")

# 順伝播テスト
x = np.random.randn(5, 3)  # バッチサイズ5, 入力次元3
y = mlp.forward(x)
print(f"\n入力: {x.shape} → 出力: {y.shape}")

### 6.2 完全な順伝播・逆伝播のテスト

In [ ]:
# 完全なテスト: 回帰問題
print("="*60)
print("MLP 回帰問題テスト")
print("="*60)

# データ
np.random.seed(42)
N = 10  # サンプル数
X = np.random.randn(N, 2)
t = np.random.randn(N, 1)  # ターゲット

# モデル
model = MLP([2, 4, 1], activation='sigmoid')
loss_fn = MSELoss()

# 順伝播
y = model.forward(X)
loss = loss_fn.forward(y, t)
print(f"\n【順伝播】")
print(f"入力 X: {X.shape}")
print(f"予測 y: {y.shape}")
print(f"損失 L: {loss:.6f}")

# 逆伝播
dout = loss_fn.backward()
model.backward(dout)
print(f"\n【逆伝播】")
for layer, name, grad in model.get_grads():
    if grad is not None:
        print(f"{layer.__class__.__name__}.{name}: {grad.shape}")

---

## 7. 勾配チェック

### 7.1 数値微分による検証

In [ ]:
def numerical_gradient_check(model, loss_fn, X, t, h=1e-5):
    """
    数値微分と解析的勾配を比較して検証
    """
    print("\n【勾配チェック】")
    print("="*70)
    
    # 解析的勾配を計算
    y = model.forward(X)
    loss = loss_fn.forward(y, t)
    dout = loss_fn.backward()
    model.backward(dout)
    
    all_passed = True
    
    for layer, name, param in model.get_params():
        analytical_grad = layer.grads[name]
        numerical_grad = np.zeros_like(param)
        
        # 数値微分
        it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            original = param[idx]
            
            # +h
            param[idx] = original + h
            y_plus = model.forward(X)
            loss_plus = loss_fn.forward(y_plus, t)
            
            # -h
            param[idx] = original - h
            y_minus = model.forward(X)
            loss_minus = loss_fn.forward(y_minus, t)
            
            # 数値勾配
            numerical_grad[idx] = (loss_plus - loss_minus) / (2 * h)
            
            # 元に戻す
            param[idx] = original
            it.iternext()
        
        # 相対誤差
        diff = np.abs(analytical_grad - numerical_grad)
        denom = np.maximum(np.abs(analytical_grad), np.abs(numerical_grad)) + 1e-10
        relative_error = np.max(diff / denom)
        
        passed = relative_error < 1e-4
        all_passed = all_passed and passed
        status = "✓ OK" if passed else "✗ NG"
        
        print(f"{layer.__class__.__name__}.{name}: 相対誤差 = {relative_error:.2e} {status}")
    
    print("="*70)
    if all_passed:
        print("全パラメータで勾配チェックに合格しました！")
    else:
        print("警告: 一部のパラメータで勾配が一致しません")
    
    return all_passed


# 勾配チェック実行
np.random.seed(123)
model = MLP([2, 3, 1], activation='sigmoid')
loss_fn = MSELoss()
X = np.random.randn(5, 2)
t = np.random.randn(5, 1)

numerical_gradient_check(model, loss_fn, X, t)

---

## 8. 演習問題

### 演習 8.1: Softmax + CrossEntropyでの勾配チェック

分類問題用のモデルを構築し、勾配チェックを実行してください。

In [ ]:
# 演習 8.1: 解答欄

# 分類問題用モデル
# TODO: MLP([2, 4, 3]) を構築（3クラス分類）
# TODO: SoftmaxCrossEntropy損失で勾配チェック

pass

### 演習 8.2: より深いネットワーク

4層のMLPを構築し、勾配が正しく逆伝播することを確認してください。

In [ ]:
# 演習 8.2: 解答欄

# TODO: MLP([3, 8, 8, 4, 1]) を構築
# TODO: 勾配チェック

pass

---

## 9. まとめと次のステップ

### このノートブックで学んだこと

1. **バッチ処理の次元規約**: X(N, D_in), W(D_in, D_out), Y(N, D_out)

2. **線形層の勾配**:
   - $\frac{\partial L}{\partial W} = X^T \frac{\partial L}{\partial Y}$
   - $\frac{\partial L}{\partial b} = \sum \frac{\partial L}{\partial Y}$
   - $\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} W^T$

3. **活性化層の勾配**: 要素ごとの微分をそのまま適用

4. **損失関数の勾配**: MSE, CrossEntropy

5. **MLP**: 複数の層を連結した多層パーセプトロン

### 次のノートブック（75: 学習ループ）への橋渡し

逆伝播で勾配が計算できるようになりました。次のノートブックでは：

- **SGD（確率的勾配降下法）** の実装
- **学習ループ** の完成
- 実際にXOR問題やMNISTを学習